# Basic API usage example

## Overview

The goal of this tutorial is to wire up a simple model that is comprised of a spatially embedded brain area, specify a simple learning objective, and optimize model parameters.

Before we begin, here are a few notes pertaining to the nomenclature we have adopted.

- A neuron ***class*** is defined by its synaptic affiliation. In `torch-biopl` you can configure types to be `Excitatory`/`Inhibitory` (where synapses have a postive/negative sign), or `Hybrid` which defaults to standard machine learning-style synapses that are unconstrained.
- Within each neuron class, you can instantiate neuron ***types***. We employ the definition of neuron types with an eye to be able to specify inter-type local connectivity rules. 
- Within each neuron type, are ***subtypes***. Neurons within a subtype can (but don't have to) share properties like time constants, nonlinearities, and biases.
- Each neural area can be configured independently by specifying its classes, types, subtypes, and inter-type connectivity rules.
- Areas can be stitched together to form larger networks.
- Learnable parameters in `torch-biopl` are usually the synaptic weights, neural time constants, and biases.

In [6]:
import torch
import torchvision.transforms as T
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm
import numpy as np

from bioplnn.models import SpatiallyEmbeddedClassifier

In [2]:
# Torch setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")

Let us wire up a simple one-area network with two neural classes. Let one of them be an `Excitatory` cell class and one be `Inhibitory`. 

In [8]:
# Model setup
model = SpatiallyEmbeddedClassifier(
    rnn_kwargs={
        "num_areas": 1,
        "area_kwargs": [
            {
                "num_cell_types": 2,
                "num_cell_subtypes": np.array([16, 16]),
                "cell_type_class": np.array(['excitatory', 'inhibitory']),
                "inter_cell_type_connectivity": np.array([[1, 1, 0], [1, 1, 1], [1, 1, 0]]),
                "in_size": [28, 28],
                "in_channels": 1,
                "out_channels": 32,
            },
        ],
    },
    num_classes = 10,
    fc_dim = 256,
    dropout = 0.5,
).to(device)

IndexError: invalid index to scalar variable.

In [ ]:
# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [4]:
# Dataloader setup
transform = T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,))])
train_data = MNIST(root="data", train=True, transform=transform)
train_loader = DataLoader(
    train_data, batch_size=256, num_workers=8, shuffle=True
)

In [5]:
# Define the training loop
model.train()
# Calculate accuracy
correct = 0
total = 0
cum_loss = 0
for epoch in range(10):
    for i, (x, labels) in enumerate(tqdm(train_loader)):
        x = x.to(device)
        labels = labels.to(device)
        torch._inductor.cudagraph_mark_step_begin()
        logits = model(x, num_steps=5)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate running accuracy and loss
        _, predicted = torch.max(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        cum_loss += loss.item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Loss: {cum_loss / len(train_loader):.4f}")

100%|██████████| 235/235 [00:08<00:00, 28.54it/s]


Accuracy: 20.89%
Loss: 2.0974


100%|██████████| 235/235 [00:05<00:00, 46.27it/s]


Accuracy: 23.93%
Loss: 4.0279


100%|██████████| 235/235 [00:05<00:00, 46.65it/s]


...

Accuracy: 28.42%
Loss: 15.0455


100%|██████████| 235/235 [00:04<00:00, 47.25it/s]


Accuracy: 29.06%
Loss: 16.7629


100%|██████████| 235/235 [00:04<00:00, 48.53it/s]

Accuracy: 29.64%
Loss: 18.4573
